

---

# 🧠 Session 3 - Topic 3:  
## `__code__`, `__closure__`, and `co_*` Attributes (Deep Dive)

This is about how **Python functions are represented internally**, and how **closures work under the hood**.

We’ll break it down:

- 🔍 What a function’s `__code__` attribute is
- 📦 How closures use `cell` objects and `free variables`
- 🛠️ Advanced example: modifying function bytecode
- ⚠️ When **not** to mess with this stuff

Let’s go!

---

## 🔍 1. What Is a Code Object?

Every Python function has an internal object called:

```python
function.__code__
```

This is a **`PyCodeObject`**, which contains all the **compiled information** about the function, such as:

| Attribute | Description |
|----------|-------------|
| `co_code` | Raw bytecode (what the interpreter runs) |
| `co_argcount` | Number of positional arguments |
| `co_varnames` | Local variable names |
| `co_consts` | Constants/literals used in the function |
| `co_names` | Names referenced in the function |
| `co_cellvars` | Variables that are closed over (from outer scope) |
| `co_freevars` | Variables from outer scopes that inner functions use |
| `co_stacksize` | Required stack size for execution |
| `co_filename`, `co_firstlineno` | Debugging info |

> This is what the Python interpreter uses when running your function.

---

## 🧱 2. Example Function for Inspection

Here’s the sample function we'll be inspecting:

```python
def outer(a, b):
    x = a + b       # cell variable
    y = 42          # cell variable

    def inner(z):
        return x * z + y   # captures x and y
    return inner

inner_fn = outer(2, 3)
```

### 💡 Key Concepts:
- `x` and `y` are defined in `outer()`
- But `inner()` uses them → they become **closed-over variables**
- These are stored in special **cell objects**

---

## 🧮 3. ASCII Diagram Explained

This shows the relationship between the outer and inner function:

```
outer.__code__.co_cellvars -> ('x','y')
          |
          v
inner.__code__.co_freevars -> ('x','y')
inner.__closure__          -> tuple(cell(x), cell(y))
```

### 🔍 What Each Part Means:

| Term | Meaning |
|------|---------|
| `co_cellvars` | Variables in the outer function that might be captured by nested functions |
| `co_freevars` | Variables used in the inner function but defined in an outer scope |
| `__closure__` | Tuple of **cell objects** that hold the actual values of free variables |

So:
- `outer()` defines `x` and `y` → they're **cell variables**
- `inner()` uses them → they're **free variables**
- They are passed via `__closure__` as **cell objects**

---

## 🕵️‍♂️ 4. Dump Code Attributes (What You Can See)

This part prints out the internal attributes of the function:

```python
def dump_code(fn):
    code = fn.__code__
    attrs = [
        "co_argcount", "co_posonlyargcount", "co_kwonlyargcount",
        "co_nlocals", "co_stacksize", "co_consts", "co_names",
        "co_varnames", "co_cellvars", "co_freevars"
    ]
    print(f"\nCode attributes for {fn.__name__}:")
    for a in attrs:
        print(f" {a:<18}: {getattr(code, a)}")
    print(" Disassembly:")
    dis.dis(fn)
```

You call it on `inner_fn`:

```python
dump_code(inner_fn)
```

And you get output like:

```
Code attributes for inner:
 co_argcount         : 1
 co_posonlyargcount  : 0
 co_kwonlyargcount   : 0
 co_nlocals          : 1
 co_stacksize        : 5
 co_consts           : (None, 42)
 co_names            : ('x', 'z', 'y')
 co_varnames         : ('z',)
 co_cellvars         : ()
 co_freevars         : ('x', 'y')
 Disassembly:
  6           0 LOAD_DEREF               0 (x)
              2 LOAD_FAST                0 (z)
              4 BINARY_MULTIPLY
              6 LOAD_DEREF               1 (y)
              8 BINARY_ADD
             10 RETURN_VALUE
```

This tells us:
- The function takes one argument (`z`)
- It uses two **free variables**: `x` and `y`
- It references the constant `42`
- And it uses `LOAD_DEREF` (which means: load from a closure cell)

---

## 🔁 5. Free vs Cell Variables

```python
print("\nouter.co_cellvars :", outer.__code__.co_cellvars)
print("inner.co_freevars  :", inner_fn.__code__.co_freevars)
```

### Output:
```
outer.co_cellvars : ('x', 'y')
inner.co_freevars : ('x', 'y')
```

### Summary:
- `outer` owns `x` and `y` → so they’re **cell variables**
- `inner` uses them → so they’re **free variables**

They’re linked via the `__closure__`.

---

## 🛠️ 6. Modifying a Code Object (Advanced Demo)

This part is **very advanced** and not for everyday use.

It does this:
- Takes the original code object of `inner`
- Changes the literal `42` to `99` in `co_consts`
- Rebuilds the function using `types.CodeType`
- Creates a new function with the same closure (so `x` and `y` still work)

```python
orig = inner_fn.__code__
consts = list(orig.co_consts)
if 42 in consts:
    consts[consts.index(42)] = 99  # change literal 42 -> 99

new_code = types.CodeType(
    orig.co_argcount,
    orig.co_posonlyargcount,
    orig.co_kwonlyargcount,
    orig.co_nlocals,
    orig.co_stacksize,
    orig.co_flags,
    orig.co_code,
    tuple(consts),
    orig.co_names,
    orig.co_varnames,
    orig.co_filename,
    orig.co_name,
    orig.co_firstlineno,
    orig.co_lnotab,
    orig.co_freevars,
    orig.co_cellvars
)

patched_inner = types.FunctionType(new_code, globals(), "patched_inner", closure=inner_fn.__closure__)
print("\npatched_inner(10) =", patched_inner(10))  # uses 99 instead of 42
```

### Output:
```
patched_inner(10) = 5*10 + 99 = 149
```

Because `x` was 5 (`2+3`) and now `y` is 99.

---

## ⚠️ 7. When NOT to Hack Code Objects

These techniques are:

| Reason | Explanation |
|--------|-------------|
| 🔒 CPython-specific | May not work in other interpreters (e.g., PyPy or MicroPython) |
| 🤯 Hard to debug | If something goes wrong, tracing bugs can be very hard |
| 🛑 Dangerous | Can crash the interpreter if done wrong |
| 🧨 Security risk | Could allow injection attacks if used improperly |
| 🧪 Only for tooling/research | Great for compilers, debuggers, or metaprogramming tools |

> So unless you're writing tools like linters, transpilers, or doing research — **don’t do this in production code!**

---

## ✅ Summary Table

| Concept | Description |
|--------|-------------|
| `__code__` | Internal compiled representation of a function |
| `co_consts` | List of constants used in the function |
| `co_varnames` | Local variable names |
| `co_cellvars` | Variables in outer scope that may be captured |
| `co_freevars` | Variables used in inner scope but defined in outer |
| `__closure__` | Tuple of cell objects holding closed-over values |
| `LOAD_DEREF` | Bytecode instruction that loads a value from a closure |
| `types.CodeType` | Class used to manually rebuild function bytecode |
| `types.FunctionType` | Used to create a new function from a code object |

---

## 🧠 Final Thought

This deep dive shows how **Python functions are more than just code blocks** — they carry around:
- Compiled bytecode
- Variable mappings
- Constant pools
- Closure state

Understanding these concepts helps you:
- Read disassembled code
- Understand how closures really work
- Build tools that manipulate or analyze Python programs

---



In [2]:
"""
Session 3 - Topic 3
===================
__code__, __closure__, and co_* Attributes (DEEP Dive)
"""

# ---------------------------------------------------
# 1. Code Object Overview
# ---------------------------------------------------
"""
A PyCodeObject stores bytecode + metadata:
  * co_code        – raw bytecode
  * co_argcount    – positional arg count
  * co_consts      – literal pool
  * co_names       – global / attr names
  * co_varnames    – local variable names
  * co_freevars    – names captured from outer scope
  * co_cellvars    – locals captured by inner funcs
  * co_stacksize   – VM stack depth needed
  * many more …
All exposed via function.__code__.
"""
# ---------------------------------------------------
# 2. Example function to inspect
# ---------------------------------------------------
def outer(a, b):
    x = a + b
    y = 42
    def inner(z):
        return x * z + y
    return inner

inner_fn = outer(2, 3)

# ---------------------------------------------------
# 3. Dump code attributes
# ---------------------------------------------------
import dis, types, sys

def dump_code(fn):
    code = fn.__code__
    keys = ["co_argcount","co_posonlyargcount","co_kwonlyargcount",
            "co_nlocals","co_stacksize","co_consts","co_names",
            "co_varnames","co_cellvars","co_freevars"]
    print("\nAttributes for", fn.__name__)
    for k in keys:
        print(f" {k:<18}: {getattr(code, k)}")
    print(" Disassembly:")
    dis.dis(fn)

dump_code(inner_fn)

# ---------------------------------------------------
# 4. Free vs cell vars demo
# ---------------------------------------------------
print("\nouter.co_cellvars:", outer.__code__.co_cellvars)
print("inner.co_freevars :", inner_fn.__code__.co_freevars)

# ---------------------------------------------------
# 5. Modify a constant inside the code object SAFELY
# ---------------------------------------------------
orig = inner_fn.__code__
consts = list(orig.co_consts)
if 42 in consts:
    consts[consts.index(42)] = 99
new_consts = tuple(consts)

# Preferred path for 3.8+
if hasattr(orig, "replace"):                         # Py 3.8+
    patched_code = orig.replace(co_consts=new_consts)
else:                                                # Very old versions
    patched_code = types.CodeType(
        orig.co_argcount,
        orig.co_posonlyargcount if sys.version_info >= (3, 8) else 0,
        orig.co_kwonlyargcount,
        orig.co_nlocals,
        orig.co_stacksize,
        orig.co_flags,
        orig.co_code,
        new_consts,
        orig.co_names,
        orig.co_varnames,
        orig.co_filename,
        orig.co_name,
        orig.co_firstlineno,
        orig.co_lnotab,
        orig.co_freevars,
        orig.co_cellvars,
    )

patched_inner = types.FunctionType(
    patched_code, globals(), "patched_inner", closure=inner_fn.__closure__
)
print("\npatched_inner(10) =", patched_inner(10))  # Uses 99 now

# ---------------------------------------------------
# END OF TOPIC 3
# ---------------------------------------------------



Attributes for inner
 co_argcount       : 1
 co_posonlyargcount: 0
 co_kwonlyargcount : 0
 co_nlocals        : 1
 co_stacksize      : 2
 co_consts         : (None,)
 co_names          : ()
 co_varnames       : ('z',)
 co_cellvars       : ()
 co_freevars       : ('x', 'y')
 Disassembly:
              0 COPY_FREE_VARS           2

 29           2 RESUME                   0

 30           4 LOAD_DEREF               1 (x)
              6 LOAD_FAST                0 (z)
              8 BINARY_OP                5 (*)
             12 LOAD_DEREF               2 (y)
             14 BINARY_OP                0 (+)
             18 RETURN_VALUE

outer.co_cellvars: ('x', 'y')
inner.co_freevars : ('x', 'y')

patched_inner(10) = 92
